## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli
# Trabajo práctico 2 : Modelo de regresión lineal del dataset Properatti
## Objetivos
El objetivo de este trabajo final es generar y comparar estadísticamente tres modelos de regresión lineal sobre el dataset limpio de Properatti construido en el TP_1; en este, fue planteado la hipótesis que el precio (variable objetivo) de las propiedades iba a estar influenciado principalmente por la superficie y la ubicación (variables predictoras). 

Con base a esto, se han planteado los siguientes objetivos específicos:
- Explorar el dataset limpio con la finalidad de verificar si debe realizarse una ultima limpieza o pueden utilizase los datos directamente;
- Realizar una visualización general de las distribuciones y relaciones del dataset con la finalidad de determinar la zona, tipo de inmueble y variables predictoras y objetivo para la realización de los modelos;
- Construir modelos de regresión lineal simple y multiple e interpretar sus metricas con la finalidad de identificar el que mejor permita obtener una predicción confiable de la variable objetivo;
- Implementar un modelo de regularización con la finalidad de compararlos y evaluar si existe o no problemas de sobreajuste;
- Determinar el modelo que más se ajusta al comportamiento de los datos analizados. 

In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

## Visualización del dataset

In [ ]:
# Visualización de la forma y atributos del dataset
data = pd.read_csv("./data/data_limpio_gdf.csv")
print(data.shape)
print("El dataset está compuesto por:", data.shape[0], "filas y",data.shape[1],"columnas.")
data.sample(5)

### Descripción de las columnas del dataset limpio:

Las columnas que incluye son:

● municipio: ubicación del inmueble por su municipio/barrios

● provincia: ubicación del inmueble por provincia

● lat  ●lon: ubicación de latitud y longitud

● superficie_m2_total: superficie total en m² del inmueble

● price_usd: Precio en dólares del inmueble

● tipo: tipo de inmueble en venta (casa, departamento, ph, tienda)

● ambientes_cat: cantidad de ambientes del inmueble (0, 1, 2, 3 , 4 o más)

● precio_usd_por_m2: Precio en dólares por metro cuadrado (USD/m²: precio dólares / superficie)

● tipo_cat_code: categoría numérica de tipo de inmueble

● municipio_cat_code: categoría numérica de municipios

● provincia_cat_code: categoría numérica de provincia

● tipo_cat_code: categoría numérica de ambientes_cat

● geometry: figura geométrica de latitud y la longitud

● country_name: nombre del país donde ocurre la operación inmobiliaria

● **precio_usd_por_m2_cat: categoría numérica de precio_usd_por_m2**
# Análisis exploratorio y visualización de correlaciones entre las variables

In [ ]:
#Revisamos la presencia de datos NaN
data.isna().sum().sort_values()
#La columna "ambientes_cat" quedó con 1248 registros nulos

In [ ]:
missing_by_row=data.isna().sum().sort_values(ascending=False)[0:6]
missing_by_row

In [ ]:
#se realiza lo siguiente solo a fines de graficar 
missing_by_row=data.isna().sum().sort_values(ascending=False)[0:6]
#se grafica cantidad de datos faltantes por fila del data set a fines practicos se visualizan solo las primeras 5 filas
sns.barplot(x=missing_by_row.index, y=missing_by_row.astype(int)) 
plt.title("Cantidad de datos faltantes por filas")
plt.xlabel("Cantidad de datos faltantes")
plt.ylabel("Registros nulos")
plt.show()


In [ ]:
#reviso donde están ubicados y a que propiedad pertenecen los registros nulos para saber si afectaran escoger un tipo de inmueble y su zona
mascara_nulos = data["ambientes_cat"].astype(str) == "nan" 
data_nulos = data[mascara_nulos]
data_nulos.loc[:, ["municipio", 'tipo', 'ambientes_cat', "precio_usd"]].sample(7)
#print(data[mascara_nulos].index)

In [ ]:
#se realiza lo siguiente solo a fines de graficar en presentacion.
missing_by_row_2= data_nulos.groupby('municipio')['tipo'].count().sort_values(ascending=False).head(25)
missing_by_row_2_porc= missing_by_row_2/data['ambientes_cat'].isna().sum()*100
pareto=missing_by_row_2_porc.values
acum=[]
val_acum=0
for i in missing_by_row_2_porc:
    val_acum= val_acum+i
    acum.append(val_acum)
pareto=acum
pareto
# print(data['municipio'].unique().shape)

# #Revisamos la distribución de los nulos por municipio
fig=plt.figure()
ax= fig.add_subplot(1,1,1)
ax.set_title('Pareto Municipios')
ax.bar(missing_by_row_2.index, missing_by_row_2, color="C0")
ax2=ax.twinx()
ax2.plot(missing_by_row_2.index,pareto,color="C1",marker="D",ms=5)
# ax2.yaxis.set_major_formatter(PercentFormatter(2))
ax.tick_params(axis="y", colors="C0")
ax2.tick_params(axis="y", colors="C1")
ax.set_xticklabels(missing_by_row_2.index, rotation=90)

plt.show()


In [ ]:
#agrupamos los registros donde hay nulos (solo para explicar que no tiene impacto la eliminación de los registros)
pd.options.display.max_rows = None
data_nulos.groupby(["tipo"])["municipio"].value_counts().sort_values(ascending=False)
#vemos que los nan están distribuidos equitativamente y no están concentrados en una mismo municipio

In [ ]:
#Los elimino 
data.dropna(subset=['ambientes_cat'], inplace=True)
print(data.isna().sum())

In [ ]:
data.describe()
#existen datos que no permiten ver los estadísticos ya que hay valores de 0 en sup_m2_total e inf en precio_usd_por_m2: eliminarlos

In [ ]:
#eliminamos del dataset los registros de sup_m2_total con valores de cero
data.drop(data[(data["sup_m2_total"] ==0)].index, inplace=True ,axis=0)

In [ ]:
#Realizamos una descripción estadística del dataset
data.describe()
#Puede observarse mejor los estadisticos media, desv estandar y los minimos y maximos

In [ ]:
#graficamos las provincias y municipios que contengan un valor mínimo de 500 registros por municipio (para una mejor visualización)
limite = 500
data = data.copy().groupby(['municipio']).filter(lambda grp: grp.shape[0] > limite)

In [ ]:
fig= plt.subplots(figsize=(20,20),constrained_layout=True)
grid = gridspec.GridSpec(2, 1, height_ratios=[1, 3])

ax1=plt.subplot(grid[0])
sns.countplot(data=data,y="provincia",order=data["provincia"].value_counts().index ,ax=ax1,color="g")

ax1.set_yticklabels(ax1.get_yticklabels(),fontsize="medium")
ax1.set_title("Distribucion de registros segun la provincia", fontsize= 'large')

ax2=plt.subplot(grid[1])
sns.countplot(data=data,x="municipio",order=data["municipio"].value_counts().index,ax=ax2,color="b")


ax2.set_title("Distribucion de registros segun los municipios", fontsize= 'large')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=90,ha="right")
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
ax1.grid()
ax2.grid()
plt.show()

La mayor cantidad de registros están Capital Federal para los barrios de Palermo, Belgrano, Caballito.
Consideraremos Capital Federal para la evaluación de los modelos 

In [ ]:
#Revisamos la distribución de registros por tipo de inmueble
plt.figure(figsize=(5,3))

plt.gca().yaxis.set_major_formatter(ScalarFormatter())
ax = sns.countplot(data = data, x = "tipo")
ax.set_xticklabels(ax.get_xticklabels(),rotation=40,ha="right")
plt.show()

#Apartamentos tiene la mayoría de los datos

Apartamentos puede ser una buena eleccion para la evaluacion de los modelos

In [ ]:
#armamos un dataset nuevo seleccionando capital federal y apartamentos
data=data.copy()
condicion_provincia= data["provincia"]=="Capital Federal"
condicion_tipo= data["tipo"]== 'apartment'
condicion_compuesta= condicion_provincia&condicion_tipo
data = data[condicion_compuesta]
data.shape

## Correlación entre la variables del dataset

In [ ]:
#Vemos la correlación entre las variables 
data_corr = data.corr()
#graficamos
plt.figure(figsize=(6,6))
sns.heatmap(data_corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlación entre variables")
plt.show()


De este cuadro podemos detectar una correlación significativa entre:
*precio_usd y sup_m2_total
*precio_usd y ambientes_cat_code
#precio_usd y lat
variables que utilizaremos para las predicciones

In [ ]:
#analizamos la correlación entre cada una de las variables.
figz= plt.figure()
mask_cols= ["sup_m2_total","precio_usd","precio_usd_por_m2", "ambientes_cat", "municipio_cat_code", "ambientes_cat_code","lat"]
graph=sns.pairplot(data[mask_cols],hue="sup_m2_total")
graph.fig.set_size_inches(20,10)
plt.grid()
plt.show()


In [ ]:
# Realizamos los siguientes gráficos para visualizar mejor (zoom) las relaciones- En primera medida analizamos metros totales con precio en dolares
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "sup_m2_total", "precio_usd", alpha=.5)
g.add_legend()


In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "sup_m2_total", "precio_usd_por_m2", alpha=.5)
g.add_legend()

De estos dos graficos vemos mejor relacion entre variables se enuentra entre sup_m2_total y precio_usd por lo cual utilizaremos estas dos features para predecir

Ademas se puede detectar un outlier que podria impactar sobre las predicciones en la variable Superficie para valores mayores a 965m2 y uno para precios 4x10e6. los cuales eliminaremos

In [ ]:
#detectamos el valor maximo de superficie total
data["sup_m2_total"].max()

In [ ]:
#eliminamos los outliers
data.drop(data[(data["sup_m2_total"]>=965)].index, inplace=True ,axis=0)

In [ ]:
#corroboramos que se eliminó el outlier de superficie
data["sup_m2_total"].max()

In [ ]:
#corroboramos que se eliminó el outlier de superficie
data.shape

In [ ]:
#eliminamos outliers para valores de propiedades mayores a 4M usd
outliers_precios= data["precio_usd"]>=4000000
data.drop(data[outliers_precios].index, inplace=True ,axis=0)

In [ ]:
#corroboro que elimino los outliers
data.shape

Conclusiones del análisis de variables predictoras y target:
- La mayor cantidad de registros están Capital Federal.
- Utilizaremos departamentos como el tipo de inmueble a modelar por contener una mayor cantidad de datos
- Consideraremos como variables predictora Superficie total y variable objetivo precio usd por su alta correlación, y su distribución. Luego evaluaremos el impacto de las variables de ubicación y cantidad de ambientes.


## Regresión lineal simple

Analizaremos la relacion existente entre la variable objetivo precio total en dolares y su feature la superficie total

In [ ]:
#preparamos la matriz de features y target
X = data[['sup_m2_total']]
y = data['precio_usd']

# Instanciamos el modelo.
lm = linear_model.LinearRegression()

# Dividimos el dataset en train y test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustamos el modelo a los datos de entrenamiento
model = lm.fit(Xtrain, ytrain)

# Predecimos etiquetas para datos desconocidos.
y_pred = lm.predict(Xtest)


In [ ]:
# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto =', ' ', model.intercept_)
print ('b_sup_m2_total=', ' ', model.coef_)
# imprimimos la métrica que mide la bondad de ajusto del modelo. En este caso el R2.
print("R2_train: ", model.score(Xtrain, ytrain))
print ('R2_test=','', metrics.r2_score(ytest, y_pred))
print ('MSE:', metrics.mean_squared_error(ytest, y_pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(ytest, y_pred)))


In [ ]:
# Graficamos el modelo re regresion del modelo con train_test_split
plt.figure(figsize=(10,10))
plt.plot(Xtest,y_pred,color="red",label="Predict line")
plt.scatter(X,y)
plt.xlabel("Superficie m2 total")
plt.ylabel("Precio usd")
plt.title('Modelo VS Precios Reales')
plt.show()

Resumen de los parametros estadisticos determinados:

El valor inicial de las propiedades es de 25380 u$s, e incrementa 3480 u$s por metro cuadrado de superficie que tiene el departamento en Capital Federal.

R2 mide lo bien que un modelo de regresión se ajusta a los datos reales. En otras palabras, se trata de una medida de la precisión general del modelo. vemos que nuestro modelo tiene una precisión del 67% con los datos de testeos. O sea, que el 67% de los datos de precio usd es predicha por la variable superficie m2 total

## Regresión lineal múltiple

Agregamos a la correlacion las variables latitud y longitud

In [ ]:
# Asignamos las variables predictoras
X = data[['lon','lat', 'sup_m2_total']]
y = data['precio_usd']

# Normalizamos los datos
scaler = StandardScaler()
scaler.fit(X)
Xscaler = scaler.fit_transform(X)

# Dividimos en train y test
Xtrain_regmul, Xtest_regmul, ytrain_regmul, ytest_regmul = train_test_split(Xscaler, y, test_size=0.2, random_state=42)

# Instanciamos el modelo y lo entrenamos
lr= linear_model.LinearRegression()
model=lr.fit(Xtrain_regmul,ytrain_regmul)


In [ ]:
# Predecimos etiquetas para datos desconocidos.
y_pred_regmul = lr.predict(Xtest_regmul)

In [ ]:
# Vemos los coeficientes 
print('Coeficientes: ', lr.coef_)
print('Intercepto: ', lr.intercept_)
print("R2_train: ", model.score(Xtrain_regmul,ytrain_regmul))
print ('R2_test:', metrics.r2_score(ytest_regmul, y_pred_regmul))
print ('MSE:', metrics.mean_squared_error(ytest_regmul, y_pred_regmul))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(ytest_regmul, y_pred_regmul)))

vemos que al agregar a la correlación las variables de ubicación (lon y lat) el modelo de predicción mejora en 0.02%

In [ ]:
rmse_simple= np.sqrt(metrics.mean_squared_error(ytest, y_pred))
rmse_multiple= np.sqrt(metrics.mean_squared_error(ytest_regmul, y_pred_regmul))
print(rmse_simple)
print(rmse_multiple)

In [ ]:
print('Diferencia porcentual entre el rmse de la regresión simple y la regresión múltiple: ', ((rmse_multiple - rmse_simple)/rmse_simple)*100)
print('Diferencia absoluta entre el rmse de la regresión simple y la regresión múltiple: ', rmse_multiple - rmse_simple)

Con estas lineas podemos comprobar que al agregar dos variables adicionales, el modelo predice mejor, disminuyendo el error cuadratico medio en un 2%.

In [ ]:
# Modelamos con statsmodels

X_train_sm = sm.add_constant(Xtrain_regmul)

model = sm.OLS(ytrain_regmul, X_train_sm).fit()

print(model.summary())

Comparando la regresión multiple sencilla con la regresión OLS podemos observar como OLS nos permite validar la significancia de los datos obtenidos. Vemos con los p values para las variables latitud y longitud son menores que el nivel de significancia por lo cual estas variables están explicando o tienen valor de predicción sobre el valor de nuestra variable objetivo precio en dólares.

Como vimos en los puntos anteriores el agregar estas dos variables mejora el modelo de predicción.

## Regularización ridge y lasso

Variables predictoras: latitud, longitud y superficie total

Variable objetivo: precio en usd

Volvemos a presentar las metricas obtenidas en la regresión multiple con OLS para las variables mencionadas para comparar con las metricas obtenidas en Lasso y Ridge

Ridge

In [ ]:
# Probamos con regularización ridge
lm_ridge = linear_model.RidgeCV(alphas=np.logspace(-10, 3, 200))

model_ridge = lm_ridge.fit(Xtrain_regmul, ytrain_regmul)

print ('Modelo Ridge:')
print('hiperparametro alpha: ', model_ridge.alpha_)
print('coeficientes ajustados: ', model_ridge.coef_)
print('intercepto: ', model_ridge.intercept_)
print('R2 train: ', model_ridge.score(Xtrain_regmul, ytrain_regmul))
print('R2 test: ', model_ridge.score(Xtest_regmul, ytest_regmul))

Lasso

In [ ]:
# Probamos con regularización lasso

lm_lasso = linear_model.LassoCV(alphas=np.logspace(-10, 3, 200), cv=5, tol=0.01)

model_lasso = lm_lasso.fit(Xtrain_regmul, ytrain_regmul)

print ('Modelo Lasso:')
print('hiperparametro alpha : ', model_lasso.alpha_)
print('coeficientes ajustados: ', model_lasso.coef_)
print('intercepto: ', model_lasso.intercept_)
print('R2 train: ', model_lasso.score(Xtrain_regmul, ytrain_regmul))
print('R2 test: ', model_lasso.score(Xtest_regmul, ytest_regmul))


Vemos que ambos modelos de regularización dan igual R2 que el modelo de regresión lineal multiple, entendiendo que el modelo de regresion lineal multiple no tenia overfitting por corregir.

Lasso disminuye los valores betas de ubicacion (lon y lat) y da mas peso a superficie, respecto a ridge, entendiendo de ello que superficie es una variable mas explicativa del peso usd.

### Por último, analizaremos el impacto de la variables ambientes en la predicción de la variable precio. 

In [ ]:
# Creamos las variables dummies para la variable categórica de ambientes
data_ambientes_dummies = pd.get_dummies(data, columns=['ambientes_cat'], drop_first=True)
print(data_ambientes_dummies.shape)
data_ambientes_dummies.head()

In [ ]:
# Asignamos las variables predictoras

X_cat = data_ambientes_dummies [['ambientes_cat_1', 'ambientes_cat_2', 'ambientes_cat_3', 'ambientes_cat_4 o mas','sup_m2_total']]
y = data_ambientes_dummies ['precio_usd']

# Normalizamos los datos
scaler = StandardScaler()
scaler.fit(X_cat)
X_cat = scaler.transform(X_cat)
# X_cat

# Dividimos en train y test
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y, test_size=0.2, random_state=42)
#y_test_cat

# Instanciamos el modelo y lo entrenamos
lr_cat = linear_model.LinearRegression()
lr_cat.fit(X_train_cat, y_train_cat)

In [ ]:
# Modelamos con statsmodels

X_train_sm_2 = sm.add_constant(X_train_cat)
model_stats = sm.OLS(y_train_cat, X_train_sm_2).fit()

print(model_stats.summary())

In [ ]:
# Probamos con regularización ridge

lm_ridge_2 = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13, 20,30))

model_ridge_2 = lm_ridge_2.fit(X_train_cat, y_train_cat)

print(lm_ridge_2.alpha_)
print(lm_ridge_2.coef_)
print(model_ridge_2.score(X_test_cat, y_test_cat))

In [ ]:
# Probamos con regularización lasso

lm_lasso_2 = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13,20,30), cv=5, tol=0.1)

model_lasso_2 = lm_lasso_2.fit(X_train_cat, y_train_cat)

print(lm_lasso_2.alpha_)
print(lm_lasso_2.coef_)
print(model_lasso_2.score(X_test_cat, y_test_cat))


Al modelar las categorias ambientes obtenemos:

- OLS: Para un nivel de confianza de 0.05, los p-value del modelo para las variables ambientes (1 a 2 ambientes) son mucho mayores al nivel de confianza por lo que se puede inferir que no tiene correlación con la variable objetivo, para 3 o mas de 4 ambientes y superficie vemos que existe correlación. F-test es mucho mayor que 0,05 por lo que podemos indicar que el modelo no es estadísticamente significativo y que por tanto las variables independientes no explican la variable dependiente. el R2 e menor que cuando no se tienen las varaibles agregadas, motivo por le cual, se podemos decir que estas variables no impactan positivamente sobre el modelo de predicción.

- Ridge:vemos que la influencia sobre el modelo de los predictores ambientes están muy poco relacionados con la variable respuesta por lo que los vemos disminuidos.

- Lasso: vemos que la influencia sobre el modelo de los predictores ambientes están muy poco relacionados con la variable respuesta por lo que los vemos disminuidos.  La mayor respuesta para explicar el precio usd de las propiedades es la superficie total